In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab11.ipynb")

# Lab 11: Resampling and the Bootstrap

Welcome to Lab 11! This covers the topics listed below: 

- Simulation (see [CIT 9.3](https://inferentialthinking.com/chapters/09/3/Simulation.html))
- Sampling (see [CIT 10](https://inferentialthinking.com/chapters/10/Sampling_and_Empirical_Distributions.html))
- Bootstrapping (see [CIT 13](https://inferentialthinking.com/chapters/13/Estimation.html))


The British Royal Air Force wanted to know how many warplanes the Germans had (some number `N`, which is a *parameter*), and they needed to estimate that quantity knowing only a random sample of the planes' serial numbers (from 1 to `N`). We know that the German's warplanes are labeled consecutively from 1 to `N`, so `N` would be the total number of warplanes they have. 

We normally investigate the random variation among our estimates by simulating a sampling procedure from the population many times and computing estimates from each sample that we generate.  In real life, if the RAF had known what the population looked like, they would have known `N` and would not have had any reason to think about random sampling. However, they didn't know what the population looked like, so they couldn't have run the simulations that we normally do.  

Simulating a sampling procedure many times was a useful exercise in *understanding random variation* for an estimate, but it's not as useful as a tool for practical data analysis.

Let's flip that sampling idea on its head to make it practical. Given *just* a random sample of serial numbers, we'll estimate `N`, and then we'll use simulation to find out how accurate our estimate probably is, without ever looking at the whole population.  This is an example of *statistical inference*.

As usual, **run the cell below** to prepare the lab and the automatic tests.

In [ ]:
# Run this cell to set up the notebook, but please don't change it.

# These lines import the Numpy and Datascience modules.
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', (FutureWarning, np.VisibleDeprecationWarning))

import d8error

## 1. Preliminaries

We (the RAF in World War II) want to know the number of warplanes fielded by the Germans.  That number is `N`.  The warplanes have serial numbers from 1 to `N`, so `N` is also equal to the largest serial number on any of the warplanes.

We only see a small number of serial numbers (assumed to be a random sample with replacement from among all the serial numbers), so we have to use estimation.

**Question 1.1**  Is `N` a population parameter or a statistic? If we compute a number using our random sample that's an estimate of `N`, is that a population parameter or a statistic? Assign either 1, 2, 3, or 4 to the variable q1_1 below.

1. `N` is a population parameter. An estimate of `N` from our random sample is a population parameter.
2. `N` is a population parameter. An estimate of `N` from our random sample is a statistic.
3. `N` is a statistic. An estimate of `N` from our random sample is a population parameter.
4. `N` is a statistic. An estimate of `N` from our random sample is a statistic.

In [ ]:
q1_1 = ...

In [ ]:
grader.check("q11")

To make the situation realistic, we're going to hide the true number of warplanes from you.  You'll have access only to this random sample:

In [ ]:
observations = Table.read_table("serial_numbers.csv")
num_observations = observations.num_rows
observations

**Question 1.2** Since we are trying to estimate the population maximum, `N`, a natural statistic to use is the sample **maximum**. In other words, we can estimate the total number of tanks using the biggest serial number in our sample.

Below, complete the implementation of the function `calculate_max_based_estimate`, which computes that statistic on an array of serial numbers. It should take as its argument an array of serial numbers and return their maximum.

After that, use it to compute an estimate of `N` using the serial numbers in `observations`. Call the estimate `max_based_estimate`.

In [ ]:
def calculate_max_based_estimate(nums):
    ...

max_based_estimate = ...
max_based_estimate

In [ ]:
grader.check("q12")

**Question 1.3** Another way to estimate `N` is to take **twice the mean** of the serial numbers in our sample. This is based on the idea that the mean of a random sample of the numbers `1` through `N` usually falls about halfway between `1` and `N`. So we can estimate `N` by doubling this mean.

Below, write a function called `calculate_mean_based_estimate` that computes that statistic. It should take as its argument a Series of serial numbers and return twice their mean.

After that, use it to compute an estimate of `N` using the serial numbers in `observations`. Call the estimate `mean_based_estimate`.

In [ ]:
def calculate_mean_based_estimate(nums):
    ...

mean_based_estimate = ...
mean_based_estimate

In [ ]:
grader.check("q13")

**Question 1.4** Look at the values of the `max_based_estimate` and `mean_based_estimate` that we get for our dataset.

In [ ]:
max_based_estimate

In [ ]:
mean_based_estimate

The value of `max_based_estimate` tells you something about `mean_based_estimate`.  Could our current `mean_based_estimate` possibly be equal to `N` (at least if we round it to the nearest integer)?  If not, is it definitely higher, definitely lower, or can we not tell?  Assign one of the choices (1-6) to the variable `q1_4` below. 
1. Yes, our `mean_based_estimate` for this sample could equal `N`.
2. No, our `mean_based_estimate` for this sample cannot be equal to `N`, it is definitely lower by roughly 3.
3. No, our `mean_based_estimate` for this sample cannot be equal to `N`, it is definitely lower by at least 12.
4. No, our `mean_based_estimate` for this sample cannot be equal to `N`, it is definitely higher by roughly 3.
5. No, our `mean_based_estimate` for this sample cannot be equal to `N`, it is definitely higher by at least 12.
6. No, our `mean_based_estimate` for this sample cannot be equal to `N`, but we cannot tell if it is lower or higher.

In [ ]:
q1_4 = ...

In [ ]:
grader.check("q14")

We can't just confidently proclaim that `max_estimate` or `mean_based_estimate` is equal to `N`.  What if we're really far off?  So we want to get a sense of the accuracy of our estimates.

## 2. Resampling 

To do this, we'll use resampling.  That is, we won't exactly simulate the observations the RAF would have really seen.  Rather we sample from our current sample, or "resample."

Why does that make any sense?

When we tried to find the value of `N`, we ideally would like to use the whole population.  Since we only have a sample, we used that to estimate `N` instead.

This time, we would like to use the population of serial numbers to draw more samples and run a simulation about estimates of `N`.  But we still only have our sample.  So, we **use our sample in place of the population** to run the simulation. We resample from our original sample with replacement as many times as there are elements in the original sample. This resampling technique is called *bootstrapping*. 

Note that in order for bootstrapping to work well, you must start with a large, random sample. Then the Law of Averages says that with high probability, your sample is representative of the population.

**Question 2.1** Write a function called `simulate_resample`.  It should generate a resample from the observed serial numbers in `observations` and return that resample.  (The resample should be a table like `observations`.)  It should take no arguments.

*Hint:* use `sample()` method.

In [ ]:
def simulate_resample():
    ...

# This is a little magic to make sure that you see the same results we did – don't change it.
np.random.seed(123)

# Don't change the line below!
one_resample = simulate_resample()
one_resample

In [ ]:
grader.check("q21")

We'll use many resamples at once to see what estimates typically look like.  However, we don't often pay attention to single resamples, so it's easy to misunderstand them.  Let's first answer some questions about our resample.

**Question 2.2** Which of the following statements are true?

1. The resample can contain serial numbers that are not in the original sample.
2. The original sample can contain serial numbers that are not in the resample.
3. The resample has either zero, one, or more than one copy of each serial number.
4. The original sample has exactly one copy of each serial number.

Assign `true_statements` to a list of the number(s) corresponding to correct statements.


In [ ]:
true_statements = ...

In [ ]:
grader.check("q22")

Now let's write a function to do many resamples at once.

Since resampling from a sample looks just like sampling from a population, the code should look almost the same.  That means we can write a function that simulates the process of either sampling from a population or resampling from a sample.  If we pass in population as its argument, it will do the former; if we pass in a sample, it will do the latter.




**Question 2.3**
Write a function called `bootstrap_estimates`.  It should take 4 arguments:
1. `original_table`: A table from which the data should be sampled.  The table will have one column named `serial number`.
2. `sample_size`: The size of each sample from that table, an integer.  
    - For example, to bootstrap, we would pass in the number of rows in the table.
3. `statistic`: A *function* that takes in an array of serial numbers as its argument and computes a statistic from the array (i.e. returns a calculated number). 
4. `num_replicates`: The number of replications to perform.

The function should simulate many samples **with replacement** from the given table.  (The number of times it does this is the 4th argument.)  

For each of those samples, it should compute the statistic on that sample. Then it should **return an array** containing each of those statistics.  The code below provides an example use of your function and describes how you can verify that you've written it correctly.

*Check your answer:* The histogram you see should be a roughly bell-shaped curve centered at 1000 with most of its area in the interval [800, 1200].

In [ ]:
def bootstrap_estimates(original_table, sample_size, statistic, num_replicates):
    ...

# Don't change the code below this comment!
# This should generate an empirical histogram of twice-mean-based estimates
# of N from samples of size 50 if N is 1000.  This should be a bell-shaped
# curve centered at 1000 with most of its mass in [800, 1200].  To verify your
# answer, make sure that's what you see!
example_estimates = bootstrap_estimates(
    Table().with_column("serial number", np.arange(1, 1000+1)),
    50,
    calculate_mean_based_estimate,
    10000)
Table().with_column("mean-based estimate", example_estimates).hist(bins=np.arange(0, 1500, 25))

In [ ]:
grader.check("q23")

Now we can go back to the sample we actually observed (the table `observations`) and estimate how much our mean-based estimate of `N` would have varied from sample to sample.

**Question 2.4** Using the bootstrap and the sample `observations`, simulate the approximate distribution of *mean-based estimates* of `N`.  Use 10,000 replications and save the estimates in an array called `bootstrap_mean_based_estimates`.  

*Hint*: Use the `bootstrap_estimates` function. 

We have provided code that plots a histogram, allowing you to visualize the simulated estimates.


In [ ]:
bootstrap_mean_based_estimates = bootstrap_estimates(observations, 
                                                     observations.num_rows, 
                                                     calculate_mean_based_estimate, 
                                                     ...

# Don't change the code below! This plots bootstrap_mean_based_estimates.
(Table()
 .with_column("mean-based estimate", bootstrap_mean_based_estimates)
 .hist(bins=np.arange(0, 200, 4)) 
)

In [ ]:
grader.check("q24")

**Question 2.5** Using the bootstrap and the sample `observations`, simulate the approximate distribution of *max estimates* of `N`.  Use 10,000 replications and save the estimates in an array called `bootstrap_max_estimates`.

We have provided code that plots a histogram, allowing you to visualize the simulated estimates.


In [ ]:
bootstrap_max_estimates = bootstrap_estimates(observations, 
                                              observations.num_rows, 
                                              calculate_max_based_estimate, 
                                              ...

# Don't change the code below! This plots bootstrap_max_estimates.
Table().with_column("max estimate", bootstrap_max_estimates).hist(bins=np.arange(0, 200, 4)) 

In [ ]:
grader.check("q25")

<!-- BEGIN QUESTION -->

**Question 2.6** `N` was actually 150! Compare the histograms of estimates you generated in 2.4 and 2.5 and answer the following questions:

1. How does the distribution of values for the mean-based estimates differ from the max estimates?
2. Which estimator should be used with the bootstrap procedure?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Computing Intervals

**Question 3.1** Compute an interval that covers the middle 95% of the mean-based bootstrap estimates.  Assign your values to `left_end_1` and `right_end_1`. 

*Hint:* Use the `percentile` function! Read up on its documentation [here](https://www.data8.org/datascience/reference-nb/datascience-reference.html#percentile()).

Verify that your interval looks like it covers 95% of the area in the histogram. The red dot on the histogram is the value of the parameter (150).

In [ ]:
left_end_1 = ...
right_end_1 = ...
print("Middle 95% of bootstrap estimates: [{:f}, {:f}]".format(left_end_1, right_end_1))

# Don't change the code below! It draws your interval and N on the histogram of mean-based estimates.
Table().with_column("mean-based estimate", bootstrap_mean_based_estimates).hist(bins=np.arange(0, 200, 4)) 
plt.plot(make_array(left_end_1, right_end_1), make_array(0, 0), color='yellow', lw=3, zorder=1)
plt.scatter(150, 0, color='red', s=30, zorder=2);

In [ ]:
grader.check("q31")

**Question 3.2** Write code that simulates the sampling and bootstrapping process again, as follows:

1. Generate a new set of random observations the RAF might have seen by sampling from the `population` table we have created for you below. Use the sample size of 70.
2. Compute an estimate of `N` from these new observations, using `calculate_mean_based_estimate`.
3. Using only the new observations, compute 10,000 bootstrap estimates of `N`.
4. Plot these bootstrap estimates and compute an interval covering the middle 95%.

In [ ]:
population = Table().with_column("serial number", np.arange(1, 150+1))

new_observations = ...
new_mean_based_estimate = ...
new_bootstrap_estimates = bootstrap_estimates(new_observations, 
                                              new_observations.num_rows,
                                              calculate_mean_based_estimate, 
                                              ...
                                              
Table().with_column("mean-based estimate", new_bootstrap_estimates).hist(bins=np.arange(0, 252, 4))
new_left_end = ...
new_right_end = ...

# Don't change code below this line!
print("New mean-based estimate: {:f}".format(new_mean_based_estimate))
print("Middle 95% of bootstrap estimates: [{:f}, {:f}]".format(new_left_end, new_right_end))

plt.plot(make_array(new_left_end, new_right_end), make_array(0, 0), color='yellow', lw=3, zorder=1)
plt.scatter(150, 0, color='red', s=30, zorder=2);

In [ ]:
grader.check("q32")

**Question 3.3** If you ran your cell above many, many times, approximately what percentage of the intervals you created would include `N` (150 in this case)? Assign either 1, 2, 3, 4, or 5 to the variable q3_3 below.

1. 100%
2. 97.5%
3. 95%
4. 5%
5. It's impossible to tell.

In [ ]:
q3_3 = ...

In [ ]:
grader.check("q33")

### With or Without Replacement?

Each time we resampled from our original sample, we sampled **with replacement**. What would happen if we tried to resample without replacement? Let's find out!

Below, we will collect another random sample of size 70 from `population` that we can then resample from. We'll call it `original_sample`.

In [ ]:
np.random.seed(23) # Magic so that you get the same result as us – don't change this line
original_sample = population.sample(70, with_replacement=False)
original_sample

**Question 3.4.** Below, 5,000 times, collect a resample of size 70 **from `original_sample` without replacement**. Compute the mean-based estimate on each resample, and store the estimates in the array `estimates_without_replacement`.

***Note:*** You **cannot** use your `bootstrap_estimates` function here, because that samples with replacement. Instead, you'll have to write a new for-loop. It's a good idea to start by copying the code from your function in  and changing the necessary pieces.

In [ ]:
estimates_without_replacement = ...
...
estimates_without_replacement

In [ ]:
grader.check("q34")

<!-- BEGIN QUESTION -->

**Question 3.5** If you completed Question 3.4 correctly, you'll notice that all 5,000 of your estimates are identical, and are equal to roughly 149.5143. Furthermore, this number is equal to the mean-based estimate derived from original_sample, without any resampling:

In [ ]:
calculate_mean_based_estimate(original_sample.column('serial number'))

Why are all of our estimates identical, and why must we sample with replacement when resampling?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## 4. Submission

**Important submission steps:** 
1. Run the tests and verify that they all pass.
2. Choose **Save Notebook** from the **File** menu, then **run the final cell**. 
3. Click the link to download the zip file.
4. Then submit the zip file to the corresponding assignment according to your instructor's directions. 

**It is your responsibility to make sure your work is saved before running the last cell.**

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)